In [1]:
import pandas as pd
import os
from tqdm import tqdm
import time
import traceback

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
drinking_survey_start = "/Users/sauravyadav/Documents/Repos/Datasets/Drinking_Surveys/After Your 1st Drink 9.11.23 FINAL.xlsx"
drinking_survey_short = "/Users/sauravyadav/Documents/Repos/Datasets/Drinking_Surveys/Drinking Follow-Ups Short 9.11.23 FINAL.xlsx"
alcohol_survey_long = "/Users/sauravyadav/Documents/Repos/Datasets/Drinking_Surveys/Drinking Followups Long 9.11.23 FINAL.xlsx"

participant_id_mapping_directory = "/Users/sauravyadav/Documents/Repos/Datasets/Drinking_Surveys/MetricWire participant ID key_FINAL.xlsx"

# column_info_mapping = "D:\Program Files\Documents\CMU\BAR_EMA\input_data\column_mapping.csv"
op_directory = '/Users/sauravyadav/Documents/Repos/Datasets/Drinking_Surveys/output'

In [3]:
ds_start = pd.read_excel(drinking_survey_start, engine='openpyxl')
ds_long = pd.read_excel(alcohol_survey_long, engine='openpyxl')
ds_short = pd.read_excel(drinking_survey_short, engine='openpyxl')

pid_mapping_df = pd.read_excel(participant_id_mapping_directory, engine='openpyxl')

### Remove garbage from cols:

### Renaming Duplicate Columns

In [4]:
for df in [ds_start, ds_long, ds_short]:
    column_name_fix = {col: col.replace('\xa0', '').strip() for col in df.columns}
    df.rename(columns=column_name_fix, inplace=True)


    new_cols = []
    seen_cols = []
    i=1
    for col in df.columns:
        if col in seen_cols:
            new_cols.append(f'{col}_{i}')
            seen_cols.append(f'{col}_{i}')
            i+=1
        else:
            new_cols.append(col)
            seen_cols.append(col)
    df.columns = new_cols

### Concatenate Morning and Random reports. Also add Participant ID

In [5]:
res_df = pd.concat([ds_start, ds_long, ds_short]).reset_index(drop=True).merge(pid_mapping_df, left_on='User Id', right_on='MW User ID', how='left').drop(columns='MW User ID').reset_index(drop=True)

In [6]:
display(res_df.head(3))

,Response Id,Response Type,User Id,Study Id,Survey Id,Source Name,Source Type,Survey Started Date,Survey Started Time,Survey Submitted Date,Survey Submitted Time,Time Zone,Trigger Id,Trigger Date,Trigger Time,Trigger Type,Trigger Name,Trigger Index,Device OS,Device OS Version,App Version,Submission Location,How long ago did you finish your drink?,How many drinks have you had so far today?,Was the drink(s) pleasurable?,Did the drink(s) relieve any unpleasant feelings or symptoms?,How intoxicated do you feel right now?,Would you like to drink more alcohol right now?,Pleaserate how you are feeling right now:Energized,Excited,Sedated,Slow thoughts,Sluggish,Up,Where are you currently?,Please specify where you are currently.,Are you with others?,How many people are you interacting with?,Who are these people you are interacting with?,Please specify who you are with currently.,How much are you enjoying this interaction?,How are you acting during this interaction?,How are you acting during this interaction?.1,How is the other person acting during this interaction?,How is the other person acting during this interaction?.1,"Are you interacting with someone virtually, on your computer, phone, or the like?",I like being alone right now.,Being alone feels good to me.,I wish I were with others right now.,I feel lonely.,I feel rejected.,I feel included.,I feel accepted.,How many people are you interacting with?.1,Who are these people you are interacting with?.1,Please specify who you are with currently..1,How much are you enjoying this interaction?.1,How are you acting during this interaction?_1,How are you acting during this interaction?_2,How is the other person acting during this interaction?_3,How is the other person acting during this interaction? .1,I like that I’m not interacting with anyone right now.,I wish I were interacting with someone right now.,I feel lonely..1,I feel rejected..1,I feel included..1,I feel accepted..1,Have you experienced the following things since your last survey?I said things without thinking.,I spent more money than I meant to.,I have felt impatient.,I made a “spur of the moment” decision.,"Since your last survey entry, how many drinks have you had?",Please state how you are feeling right now:Energized,Please state how you are feeling right now: Energized,SUB ID
0,5c787f4db05c7d1f4f74720f,Submission,58431e46e75bda1d3007c0c8,5aa83b46284c846391451166,5aa83b46284c846391451167,After Your 1st Drink,Assessment,28/02/2019,19:38:22,28/02/2019,19:39:41,-5:00,5b32a9a4d3cd426b5fc09693,28/02/2019,06:00:00,NaN,NaN,NaN,iOS,12.1.4,4.1.2,"0.00000,0.00000",1.0,1.0,80.0,73.0,0.0,79.0,77.0,92.0,13.0,21.0,19.0,86.0,1.0,CONDITION_SKIPPED,1.0,4,3,CONDITION_SKIPPED,99,100,62,100,72,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,53.0,4.0,62.0,48.0,NaN,NaN,NaN,2302
1,5c7a1b4dfcf52a1ac62adeb3,Submission,58431e46e75bda1d3007c0c8,5aa83b46284c846391451166,5aa83b46284c846391451167,After Your 1st Drink,Assessment,2019-02-03 00:00:00,00:56:27,2019-02-03 00:00:00,00:57:33,-5:00,5b32a9a4d3cd426b5fc09693,2019-01-03 00:00:00,06:00:00,NaN,NaN,NaN,iOS,12.1.4,4.1.2,"0.00000,0.00000",1.0,1.0,93.0,93.0,90.0,91.0,90.0,99.0,26.0,23.0,26.0,94.0,2.0,CONDITION_SKIPPED,1.0,5,3,CONDITION_SKIPPED,82,85,86,89,65,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,68.0,0.0,38.0,1

### Creating a mapping from current to desired columns

In [7]:
# col_mapping_df = pd.read_csv(column_info_mapping)
# value_counts = col_mapping_df['Current variable name'].value_counts().reset_index()
# value_counts = value_counts[value_counts['count'] > 1]
# display(value_counts)

In [8]:
column_mapping = {'SUB ID' : 'SubID', 'Response Id' : 'RespID', 'Response Type' : 'RespType', 'User Id' : 'UserID',
 'Source Name' : 'SurvName', 'Source Type' : 'SurvType', 'Survey Started Date' : 'InitDate',
 'Survey Started Time' : 'InitTime', 'Survey Submitted Date' : 'SubDate', 'Survey Submitted Time' : 'SubTime',
 'Trigger Date' : 'TrigDate', 'Trigger Time' : 'TrigTime',
 'How long ago did you finish your drink?' : 'UI_DrnkFin',
 'How many drinks have you had so far today?' : 'UI_DrnkNum',
 'Since your last survey entry, how many drinks have you had?' : 'DrnkNum',
 'Was the drink(s) pleasurable?' : 'Plsur',
 'Did the drink(s) relieve any unpleasant feelings or symptoms?' : 'Rliev',
 'How intoxicated do you feel right now?' : 'IntxNow',
 'Would you like to drink more alcohol right now?' : 'MorAlc',
 'Pleaserate how you are feeling right now:Energized' : 'enrgz',
 'Excited' : 'excit', 'Sedated' : 'sedat', 'Slow thoughts' : 'slotht',
 'Sluggish' : 'slug', 'Up' : 'up', 'Where are you currently?' : 'crntloc',
 'Please specify where you are currently.' : 'othrloc', 'Are you with others?' : 'wthothr',
 'How many people are you interacting with?' : 'NumOth',
 'Who are these people you are interacting with?' : 'OthTyp',
 'Please specify who you are with currently.' : 'OthWho',
 'How much are you enjoying this interaction?' : 'EnjInt',
 'How are you acting during this interaction?' : 'ActCld',
 'How are you acting during this interaction?.1' : 'ActAcc',
 'How is the other person acting during this interaction?' :
 'OthCld', 'How is the other person acting during this interaction?.1' :
 'OthAcc',
 'Are you interacting with someone virtually, on your computer, phone, or the like?' : 'Vrtl',
 'I like being alone right now.' : 'AlnLke', 'Being alone feels good to me.' : 'AlnGd',
 'I wish I were with others right now.' : 'WshOth', 'I feel lonely.' : 'lnly', 
 'I feel rejected.' : 'rjct', 'I feel included.' : 'incl', 'I feel accepted.' : 'acpt',
 'How many people are you interacting with?.1' : 'vrtint',
 'Who are these people you are interacting with?.1' : 'vrtoth',
 'Please specify who you are with currently..1' : 'vrtwho', 
 'How much are you enjoying this interaction?.1' : 'vrtenj', 
 'How are you acting during this interaction?_1' : 'vrtcld', 
 'How are you acting during this interaction?_2' : 'VrtAcc', 
 'How is the other person acting during this interaction?_3' : 'VrtOthCld', 
 'How is the other person acting during this interaction? .1' : 'VrtOthAcc', 
 'I like that I’m not interacting with anyone right now.' : 'NotInt', 
 'I wish I were interacting with someone right now.' : 'WshInt', 
 'I feel lonely..1' : 'UI_NtInt_lnly', 'I feel rejected..1' : 'UI_NtInt_rjct', 
 'I feel included..1' : 'UI_NtInt_incl', 'I feel accepted..1' : 'UI_NtInt_acpt', 
 'Have you experienced the following things since your last survey?I said things without thinking.' : 'said', 
 'I spent more money than I meant to.' : 'spent', 'I have felt impatient.' : 'impt', 
 'I made a “spur of the moment” decision.' : 'spur'}

### Renaming & Reordering Columns to Desired Column Names

In [9]:
res_df = res_df[column_mapping.keys()]
res_df = res_df.rename(columns = column_mapping)

In [10]:
res_df.sample()

,SubID,RespID,RespType,UserID,SurvName,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,UI_DrnkFin,UI_DrnkNum,DrnkNum,Plsur,Rliev,IntxNow,MorAlc,enrgz,excit,sedat,slotht,slug,up,crntloc,othrloc,wthothr,NumOth,OthTyp,OthWho,EnjInt,ActCld,ActAcc,OthCld,OthAcc,Vrtl,AlnLke,AlnGd,WshOth,lnly,rjct,incl,acpt,vrtint,vrtoth,vrtwho,vrtenj,vrtcld,VrtAcc,VrtOthCld,VrtOthAcc,NotInt,WshInt,UI_NtInt_lnly,UI_NtInt_rjct,UI_NtInt_incl,UI_NtInt_acpt,said,spent,impt,spur
38897,2341,2643df99-6b71-487a-80ac-933541818bcd,Missed,5cb60d54c36a86700d9a4971,Drinking Follow-Ups,NaN,31/05/2019,01:31:29,31/05/2019,01:41:29,31/05/2019,01:31:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
res_df.to_csv(f'{op_directory}/prelim_data_0.csv',index=False)